In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import geohash

In [2]:
import os
os.environ['PYSPARK_PYTHON'] = '/anaconda/bin/python'

In [3]:
import findspark
findspark.init()
print findspark.find()
# Depending on your setup you might have to change this line of code
#findspark makes sure I dont need the below on homebrew.
#os.environ['SPARK_HOME']="/usr/local/Cellar/apache-spark/1.5.1/libexec/"
#the below actually broke my spark, so I removed it. 
#Depending on how you started the notebook, you might need it.
#os.environ['PYSPARK_SUBMIT_ARGS']="--master local pyspark --executor-memory 4g"

/usr/local/opt/apache-spark/libexec


In [4]:
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark')
    .set("spark.driver.memory", "5g"))
sc = pyspark.SparkContext(conf=conf)

In [5]:
from pyspark.sql import SQLContext
sqlsc=SQLContext(sc)

In [6]:
raw_data = sc.textFile('yellow_tripdata_2015-02.csv').map(lambda line: tuple(line.split(','))).zipWithIndex()

In [7]:
column_names = raw_data.take(1)[0]
print column_names

((u'VendorID', u'tpep_pickup_datetime', u'tpep_dropoff_datetime', u'passenger_count', u'trip_distance', u'pickup_longitude', u'pickup_latitude', u'RateCodeID', u'store_and_fwd_flag', u'dropoff_longitude', u'dropoff_latitude', u'payment_type', u'fare_amount', u'extra', u'mta_tax', u'tip_amount', u'tolls_amount', u'improvement_surcharge', u'total_amount'), 0)


In [13]:
def col_extracter(row):
    # Helper function that strips the index and grabs the following columns:
    # u'tpep_pickup_datetime', u'tpep_dropoff_datetime', u'pickup_longitude', u'pickup_latitude', u'dropoff_longitude', u'dropoff_latitude'
    return (row[1], row[2], row[5], row[6], row[9], row[10])

def replace_coords_with_geohash(row):
    # Helper function to replace pickup and dropoff lat/long coords with a geohash for each location
    return (row[0],row[1],geohash.encode(float(row[3]),float(row[2])), geohash.encode(float(row[5]),float(row[4])))

def data_cleaner(row):
    # Helper function that calls functions to extract columns and add geohashs
    # Returns a tuple of the form: (u'tpep_pickup_datetime', u'tpep_dropoff_datetime', u'pickup_geohash', u'dropoff_geohash')
    return replace_coords_with_geohash(col_extracter(row))

In [16]:
#get rid of header row and indices and replace pickup/dropoff coords with geohashs
data = raw_data.filter(lambda (row,index): index > 0).keys().map(data_cleaner)

In [18]:
data.take(1)

[(u'2015-02-08 11:33:46',
  u'2015-02-08 11:37:45',
  'dr5rthrk629c',
  'dr5rtjx4xcf2')]